# Validators

Obsplus provides a way to declare and test certain assumptions about data. The implementation is specifically geared towards nested tree structures (like obspy's `Catalog` object) and 

<div class="alert alert-warning">

**Warning**: In the future we may move much of this functionality to obspy as described in [this proposal](https://github.com/obspy/obspy/issues/2154), but an appropriate deprecation cycle will be implemented.
</div>
